#Log extraction (Swin unsupervised)

In [ ]:
def search_string_in_file(file_name, string_to_search):
    """Search for the given string in file and return lines containing that string,
    along with line numbers"""
    line_number = 0
    list_of_results = []
    # Open the file in read only mode
    with open(file_name, 'r') as read_obj:
        # Read all lines in the file one by one
        for line in read_obj:
            # For each line, check if line contains the string
            line_number += 1
            if string_to_search in line:
                # If yes, then add the line number & line as a tuple in the list
                list_of_results.append((line_number, line.rstrip()))
    # Return list of tuples containing line numbers and lines where string is found
    return list_of_results

In [ ]:
def readline(filePath, lineNumber):
  file = open(filePath)
  content = file.readlines()
  return content[lineNumber-1]

In [ ]:
import pprint

def getMetrics(filePath):
  #Get lines for training fin to get line for avg loss during training#
  linesTrainEnd = search_string_in_file(filePath, ": INFO EPOCH ")
  epochNumbers = [index[1].split(': INFO EPOCH ')[1].split(" ")[0] for index in linesTrainEnd]
  trainLossLinesNumbers = [index[0] - 1 for index in linesTrainEnd]
  trainingLossLines = [readline(filePath, line) for line in trainLossLinesNumbers]
  trainingLossMeasures = [line.split('loss')[1].split(" (")[1].split(")")[0] for line in trainingLossLines]

  return trainingLossMeasures, epochNumbers

In [ ]:
TrainingLoss, epochNumbers = getMetrics("/content/log_rank0.txt")
print(len(TrainingLoss), len(epochNumbers))

import pandas as pd 
    
# List1 
lst = [TrainingLoss, epochNumbers]
    
df = pd.DataFrame({'Epoch':epochNumbers, 'AvgTrainingLoss':TrainingLoss})
display(df)
df.to_csv('output.csv')

300 300


,Epoch,AvgTrainingLoss
0,0,9.6069
1,1,9.0855
2,2,8.9939
3,3,8.9900
4,4,9.0401
...,...,...
295,295,9.3834
296,296,9.4073
297,297,9.3831
298,298,9.3805


#Log extraction (Swin supervised)

In [1]:
def search_string_in_file(file_name, string_to_search):
    """Search for the given string in file and return lines containing that string,
    along with line numbers"""
    line_number = 0
    list_of_results = []
    # Open the file in read only mode
    with open(file_name, 'r') as read_obj:
        # Read all lines in the file one by one
        for line in read_obj:
            # For each line, check if line contains the string
            line_number += 1
            if string_to_search in line:
                # If yes, then add the line number & line as a tuple in the list
                list_of_results.append((line_number, line.rstrip()))
    # Return list of tuples containing line numbers and lines where string is found
    return list_of_results

In [2]:
def readline(filePath, lineNumber):
  file = open(filePath)
  content = file.readlines()
  return content[lineNumber-1]

In [3]:
testString = 'Acc@1 0.000 Acc@5 0.000'

def splitAccValues(accString):
  Accsplits =accString.split(' ')
  return Accsplits

print(splitAccValues(testString))

['Acc@1', '0.000', 'Acc@5', '0.000']


In [4]:
import pprint

def getMetrics(filePath):
  #[--------------------------------------------------------] Testing accuracy measures
  linesAcc = search_string_in_file(filePath, "INFO  * Acc@1")
  # print(len(linesAcc))
  # pprint.pprint(lines)
  accuracyMeasures = [line[1].split('* ')[1] for line in linesAcc]
  # pprint.pprint(accuracyMeasures)
  Acc1List = [splitAccValues(string)[1] for string in accuracyMeasures]
  Acc5List = [splitAccValues(string)[3] for string in accuracyMeasures]

  #[--------------------------------------------------------] Testing loss
  #get loss lines from fin testing acc lines
  lossLines = [line[0] - 1 for line in linesAcc]
  # print(lossLines)

  #Get testing losses each ep
  lossLineContent = [readline(filePath, lineNumber) for lineNumber in lossLines]
  # pprint.pprint(lossLineContent)
  TestinglossValues = [content.split('Loss')[1].split(' (')[1].split(')')[0] for content in lossLineContent]
  # pprint.pprint(TestinglossValues)

  #[----------------------------------------------------------------------------] Training loss
  #Get lines for training fin to get line for avg loss during training#
  linesTrainEnd = search_string_in_file(filePath, ": INFO EPOCH ")
  epochNumbers = [index[1].split(': INFO EPOCH ')[1].split(" ")[0] for index in linesTrainEnd]
  epochNumbers.insert(0,'BaseLine')
  trainLossLinesNumbers = [index[0] - 1 for index in linesTrainEnd]
  trainingLossLines = [readline(filePath, line) for line in trainLossLinesNumbers]
  trainingLossMeasures = [line.split('loss')[1].split(" (")[1].split(")")[0] for line in trainingLossLines]
  trainingLossMeasures.insert(0,'N/a')
  # pprint.pprint(trainingLossMeasures) #50
  return epochNumbers, Acc1List, Acc5List, TestinglossValues, trainingLossMeasures

In [5]:
epochsNumbers, Acc1List, Acc5List, TestingLoss, TrainingLoss = getMetrics("/content/drive/MyDrive/Colab Notebooks/FYP/MOBY SSL SWIN/swin_tiny_patch4_window7_224/MineUnsupervisedPretrainMOBY672/log_rank0.txt")
print(len(epochsNumbers), len(Acc1List), len(Acc5List), len(TestingLoss), len(TrainingLoss))

# print(epochsNumbers, Acc1List, Acc5List, TestingLoss, TrainingLoss)
import pandas as pd 
    
# List1 
lst = [epochsNumbers, Acc1List, Acc5List, TestingLoss, TrainingLoss]
    
df = pd.DataFrame({'Epoch': epochsNumbers, 'Acc@1': Acc1List, 'Acc@5':Acc5List, 'AvgTestingLoss':TestingLoss, 'AvgTrainingLoss':TrainingLoss})
display(df)
df.to_csv('output.csv')

13 13 13 13 13


,Epoch,Acc@1,Acc@5,AvgTestingLoss,AvgTrainingLoss
0,BaseLine,7.119,100.000,6.9062,N/a
1,0,72.703,100.000,0.9962,2.9928
2,1,76.986,100.000,0.8241,1.7950
3,2,78.864,100.000,0.7714,1.7605
4,3,79.665,100.000,0.7500,1.7521
5,4,80.407,100.000,0.7122,1.7447
6,5,80.716,100.000,0.7086,1.7367
7,6,81.901,100.000,0.6810,1.7314
8,7,82.140,100.000,0.6684,1.7287
9,8,82.367,100.000,0.6624,1.7259


#Extract from list

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
values = [
"Acc@1 7.119 Acc@5 100.000",
"Acc@1 72.703 Acc@5 100.000",
"Acc@1 72.703 Acc@5 100.000",
"Acc@1 72.703 Acc@5 100.000",
"Acc@1 72.703 Acc@5 100.000",
"Acc@1 72.703 Acc@5 100.000",
"Acc@1 72.703 Acc@5 100.000",
"Acc@1 72.703 Acc@5 100.000",
"Acc@1 72.703 Acc@5 100.000",
"Acc@1 73.596 Acc@5 100.000",
"Acc@1 73.845 Acc@5 100.000",
"Acc@1 74.863 Acc@5 100.000",
"Acc@1 75.778 Acc@5 100.000",
"Acc@1 75.951 Acc@5 100.000",
"Acc@1 76.704 Acc@5 100.000",
"Acc@1 77.051 Acc@5 100.000",
"Acc@1 77.375 Acc@5 100.000",
"Acc@1 77.781 Acc@5 100.000",
"Acc@1 78.144 Acc@5 100.000",
"Acc@1 78.317 Acc@5 100.000",
"Acc@1 78.296 Acc@5 100.000",
"Acc@1 79.054 Acc@5 100.000",
"Acc@1 79.373 Acc@5 100.000",
]

Acc1List = [splitAccValues(string)[1] for string in values]
Acc5List = [splitAccValues(string)[3] for string in values]

lst = [Acc1List, Acc5List,
       TestingLoss, TrainingLoss]
    
df = pd.DataFrame({'Acc@1': Acc1List, 'Acc@5':Acc5List})
display(df)
df.to_csv('output.csv')

,Acc@1,Acc@5
0,7.119,100.000
1,72.703,100.000
2,72.703,100.000
3,72.703,100.000
4,72.703,100.000
5,72.703,100.000
6,72.703,100.000
7,72.703,100.000
8,72.703,100.000
9,73.596,100.000
